### 📁 데이터 로드 및 전처리

#### PDF 문서 도메인별 분류 로드

각 도메인(commerce, finance, law, medical, public)별로 PDF 문서들을 자동으로 로드하는 기능입니다. 
`data/agentic_retrieval` 폴더 하위의 각 도메인 폴더에서 PDF 파일들을 찾아 문서를 로드합니다.


In [1]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from typing import List, Dict

def load_documents_from_folders(base_path: str = "data/agentic_retrieval") -> Dict[str, List]:
    """
    data/agentic_retrieval 하위 폴더들을 순회하며 PDF 문서들을 로드합니다.
    
    Args:
        base_path: 기본 경로 (기본값: "data/agentic_retrieval")
    
    Returns:
        Dict[str, List]: 폴더명을 키로, 해당 폴더의 문서들을 값으로 하는 딕셔너리
    """
    documents_by_domain = {}
    base_dir = Path(base_path)
    
    if not base_dir.exists():
        print(f"경로가 존재하지 않습니다: {base_path}")
        return documents_by_domain
    
    # 하위 폴더들을 순회
    for folder in base_dir.iterdir():
        if folder.is_dir():
            domain_name = folder.name
            documents = []
            
            print(f"처리 중인 영역: {domain_name}")
            
            # 해당 폴더 내의 PDF 파일들을 찾아서 로드
            for pdf_file in folder.glob("*.pdf"):
                try:
                    loader = PyPDFLoader(str(pdf_file))
                    docs = loader.load()
                    documents.extend(docs)
                    print(f"  - 로드됨: {pdf_file.name} ({len(docs)} 페이지)")
                except Exception as e:
                    print(f"  - 오류 발생 ({pdf_file.name}): {e}")
            
            documents_by_domain[domain_name] = documents
            print(f"  총 {len(documents)} 문서 로드 완료\n")
    
    return documents_by_domain

# 문서 로드 실행
documents_by_domain = load_documents_from_folders()

처리 중인 영역: commerce
  - 로드됨: 2025년_유통산업_보고서_온라인쇼핑.pdf (5 페이지)
  - 로드됨: KIEP_한미_디지털_통상_현안과_정책_시사점.pdf (30 페이지)
  - 로드됨: 공정거래위원회_2025년_한눈에보는_공정거래제도.pdf (75 페이지)
  - 로드됨: 삼정KPMG_2025년_국내_디지털금융_주요이슈.pdf (42 페이지)
  총 152 문서 로드 완료

처리 중인 영역: finance
  - 로드됨: 월간_재정동향_2025년_2월호.pdf (2 페이지)
  총 2 문서 로드 완료

처리 중인 영역: law
  - 로드됨: 법무부_2025년_지역특화형_비자_운영계획.pdf (4 페이지)
  - 로드됨: 법무부_2025년_출입국정책_공존_소식지.pdf (40 페이지)
  총 44 문서 로드 완료

처리 중인 영역: medical
  - 로드됨: 질병관리청_2025년_성과관리_시행계획.pdf (274 페이지)
  - 로드됨: 질병관리청_2025년_주요업무_추진계획.pdf (7 페이지)
  - 로드됨: 질병관리청_전세계_감염병_발생_동향_2025년.pdf (20 페이지)
  총 301 문서 로드 완료

처리 중인 영역: public
  - 로드됨: 국토교통부_2025년_국민편안한일상_보도자료.pdf (18 페이지)
  - 로드됨: 국토교통부_2025년_핵심과제_실천계획.pdf (20 페이지)
  - 로드됨: 중소벤처기업부_2025년_업무보고.pdf (5 페이지)
  - 로드됨: 중소벤처기업부_2025년_주요업무_추진계획.pdf (25 페이지)
  총 68 문서 로드 완료



### 🗄️ 벡터 데이터베이스 구성

#### Qdrant 컬렉션 생성 및 문서 임베딩

도메인별로 별도의 Qdrant 컬렉션을 생성하고 하이브리드 검색(Dense + Sparse)을 위한 벡터 임베딩을 저장합니다.
BGE-M3 모델과 BM25를 조합하여 의미론적 검색과 키워드 검색을 모두 지원합니다.


In [2]:
from uuid import uuid4
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

# Qdrant 클라이언트 초기화
client = QdrantClient(host="localhost", port=6333)

# 임베딩 모델 초기화
dense_embeddings = OllamaEmbeddings(model="bge-m3")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")


# 각 영역별로 컬렉션 생성 및 문서 저장
for area in documents_by_domain:
    collection_name = f"agentic_retrieval_{area}"
    
    print(f"처리 중인 컬렉션: {collection_name}")
    
    # 컬렉션이 이미 존재하는지 확인
    try:
        collection_info = client.get_collection(collection_name)
        print(f"  - 컬렉션 {collection_name}이 이미 존재합니다. 건너뜁니다.")
        continue
    except Exception:
        # 컬렉션이 존재하지 않으면 생성
        
        # 문서 분할
        documents = documents_by_domain[area]
        if documents:
            # Qdrant 벡터스토어 생성 (Hybrid 모드)
            vectorstore = client.create_collection(
                        collection_name=collection_name,
                        vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)},
                        sparse_vectors_config={"sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
                    },
                )
            qdrant = QdrantVectorStore(
                    client=client,
                    collection_name=collection_name,
                    embedding=dense_embeddings,
                    sparse_embedding=sparse_embeddings,
                    retrieval_mode=RetrievalMode.HYBRID,
                    vector_name="dense",
                    sparse_vector_name="sparse",
                )
            qdrant.add_documents(documents)
            print(f"  - {len(documents)} 개의 청크가 {collection_name} 컬렉션에 저장됨")
        else:
            print(f"  - {area} 영역에 문서가 없습니다.")

print("모든 컬렉션 생성 완료!")

처리 중인 컬렉션: agentic_retrieval_commerce
  - 컬렉션 agentic_retrieval_commerce이 이미 존재합니다. 건너뜁니다.
처리 중인 컬렉션: agentic_retrieval_finance
  - 컬렉션 agentic_retrieval_finance이 이미 존재합니다. 건너뜁니다.
처리 중인 컬렉션: agentic_retrieval_law
  - 컬렉션 agentic_retrieval_law이 이미 존재합니다. 건너뜁니다.
처리 중인 컬렉션: agentic_retrieval_medical
  - 컬렉션 agentic_retrieval_medical이 이미 존재합니다. 건너뜁니다.
처리 중인 컬렉션: agentic_retrieval_public
  - 컬렉션 agentic_retrieval_public이 이미 존재합니다. 건너뜁니다.
모든 컬렉션 생성 완료!


### 🔧 검색 도구 생성

#### 도메인별 검색 도구 구성

각 도메인(상업, 금융, 법률, 의료, 공공)에 특화된 검색 도구를 생성합니다.
LangChain의 `create_retriever_tool`을 사용하여 에이전트가 활용할 수 있는 도구로 변환합니다.


In [3]:
from langchain_core.tools import create_retriever_tool
from typing import List

# 각 컬렉션별 검색기 도구 생성
retriever_tools = []

for area in documents_by_domain:
    collection_name = f"agentic_retrieval_{area}"
    
    # Qdrant 벡터스토어 연결
    qdrant = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings,
        sparse_embedding=sparse_embeddings,
        retrieval_mode=RetrievalMode.HYBRID,
        vector_name="dense",
        sparse_vector_name="sparse",
    )
    
    # 해당 영역의 고유한 source들 수집
    documents = documents_by_domain[area]
    unique_sources = set()
    for doc in documents:
        if hasattr(doc, 'metadata') and 'source' in doc.metadata:
            source_name = doc.metadata['source'].split("\\")[-1].split('.pdf')[-2]  # 파일명만 추출
            unique_sources.add(source_name)
    
    # source 정보를 포함한 상세한 description 생성
    sources_info = ", ".join(sorted(unique_sources)) if unique_sources else "다양한 문서"
    
    # 검색기 생성
    retriever = qdrant.as_retriever(search_kwargs={"k": 5})
    
    # create_retriever_tool을 사용하여 Tool 생성
    search_tool = create_retriever_tool(
        retriever=retriever,
        name=f"search_{area}",
        description=f"{area} 영역의 문서를 검색합니다. 이 도구는 {area}와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: {sources_info}"
    )
    
    retriever_tools.append(search_tool)
    print(f"✓ {area} 영역 검색 도구 생성 완료 (문서 수: {len(documents)}, 소스: {len(unique_sources)}개)")

print(f"\n총 {len(retriever_tools)}개의 검색 도구가 생성되었습니다:")
for tool in retriever_tools:
    print(f"  - {tool.name}: {tool.description}")

✓ commerce 영역 검색 도구 생성 완료 (문서 수: 152, 소스: 4개)
✓ finance 영역 검색 도구 생성 완료 (문서 수: 2, 소스: 1개)
✓ law 영역 검색 도구 생성 완료 (문서 수: 44, 소스: 2개)
✓ medical 영역 검색 도구 생성 완료 (문서 수: 301, 소스: 3개)
✓ public 영역 검색 도구 생성 완료 (문서 수: 68, 소스: 4개)

총 5개의 검색 도구가 생성되었습니다:
  - search_commerce: commerce 영역의 문서를 검색합니다. 이 도구는 commerce와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: 2025년_유통산업_보고서_온라인쇼핑, KIEP_한미_디지털_통상_현안과_정책_시사점, 공정거래위원회_2025년_한눈에보는_공정거래제도, 삼정KPMG_2025년_국내_디지털금융_주요이슈
  - search_finance: finance 영역의 문서를 검색합니다. 이 도구는 finance와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: 월간_재정동향_2025년_2월호
  - search_law: law 영역의 문서를 검색합니다. 이 도구는 law와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: 법무부_2025년_지역특화형_비자_운영계획, 법무부_2025년_출입국정책_공존_소식지
  - search_medical: medical 영역의 문서를 검색합니다. 이 도구는 medical와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: 질병관리청_2025년_성과관리_시행계획, 질병관리청_2025년_주요업무_추진계획, 질병관리청_전세계_감염병_발생_동향_2025년
  - search_public: public 영역의 문서를 검색합니다. 이 도구는 public와 관련된 질문에 답하기 위해 사용됩니다. 포함된 문서: 국토교통부_2025년_국민편안한일상_보도자료, 국토교통부_2025년_핵심과제_실천계획, 중소벤처기업부_2025년_업무보고, 중소벤처기업부_2025년_주요업무_추진

### 🤖 에이전트 시스템 구성

#### LangGraph REACT 에이전트 초기화

환경 변수를 로드하고 GPT-4.1를 활용한 ReAct(Reasoning + Acting) 에이전트를 생성합니다.
에이전트는 사용자 질문에 따라 적절한 도메인 검색 도구를 선택하여 답변을 생성합니다.


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

### 💬 에이전트 실행 및 테스트

#### 멀티 도메인 질의 처리

질병관리청 정책과 지역특화형 비자 관련 질문을 통해 에이전트가 medical과 law 도메인을 자동으로 선택하고
각각의 검색 도구를 활용하여 종합적인 답변을 생성하는 과정을 확인합니다.


In [6]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
# LLM 초기화
llm = ChatOllama(model="Hituzip/gemma3-tools:4b", temperature=0)
gpt = ChatOpenAI(model="gpt-4.1", temperature=0)
# create_react_agent를 사용하여 에이전트 생성
app = create_react_agent(
    model=gpt,
    tools=retriever_tools,
    # 시스템 프롬프트 설정
    prompt="""You are a helpful AI assistant. 
Use the provided search tools appropriately to answer user questions.
Use domain-specific search tools to find relevant information and provide accurate and useful answers.
If necessary, you can use multiple tools or use the same tool multiple times with different queries.
Always respond in Korean regardless of the language of the question."""
)

print("✓ LangGraph REACT 에이전트가 생성되었습니다!")
print(f"✓ 사용 가능한 도구: {[tool.name for tool in retriever_tools]}")

✓ LangGraph REACT 에이전트가 생성되었습니다!
✓ 사용 가능한 도구: ['search_commerce', 'search_finance', 'search_law', 'search_medical', 'search_public']


In [7]:
query = '질병 관리청이 올해 추진하는 주요 정책과 지역 특화형 비자 계획을 정리해서 알려주세요'
initial_state = {
    "messages": [HumanMessage(content=query)]
}

print(f"🤖 질문: {query}")
print("=" * 80)

async for chunk in app.astream(
    initial_state,
    stream_mode="values",
):
    if "messages" in chunk:
        latest_message = chunk["messages"][-1]
        
        # 도구 호출인 경우
        if hasattr(latest_message, 'tool_calls') and latest_message.tool_calls:
            for tool_call in latest_message.tool_calls:
                print(f"\n🔧 도구 호출: {tool_call['name']}")
                print(f"   검색어: {tool_call['args'].get('query', 'N/A')}")
        
        # 도구 결과인 경우
        if hasattr(latest_message, 'name') and latest_message.name:
            print(f"\n📄 검색 결과 ({latest_message.name}):")
            print("   검색 완료 ✓")

        if hasattr(latest_message, 'content') and latest_message.content:
            print(f"\n💭 AI 응답:")
            print("-" * 40)
            print(latest_message.content, end="", flush=True)
            print("-" * 40)

🤖 질문: 질병 관리청이 올해 추진하는 주요 정책과 지역 특화형 비자 계획을 정리해서 알려주세요

💭 AI 응답:
----------------------------------------
질병 관리청이 올해 추진하는 주요 정책과 지역 특화형 비자 계획을 정리해서 알려주세요----------------------------------------

🔧 도구 호출: search_medical
   검색어: 2025년 질병관리청 주요 정책

🔧 도구 호출: search_law
   검색어: 2025년 지역특화형 비자 운영계획

📄 검색 결과 (search_law):
   검색 완료 ✓

💭 AI 응답:
----------------------------------------
- 1 -
보도자료
보도시점 배포 즉시 보도배포2025. 2. 20.(목)[｢신(新) 출입국·이민정책｣ 후속 조치] "지역특화형 비자로 지역에 활력을"법무부, 2025년 지역특화형 비자 운영계획 시행□ 법무부는 지역소멸 위기를 극복하고 지역 균형발전을 도모하기 위해 2025년 지역특화형 비자 운영계획을 시행합니다. 2025년부터는 대상 지역 확대와 비자 제도 개편을 통해 더 많은 지자체에서 지역특화형 비자 제도가 시행될 수 있도록 할 예정입니다.□ 이번 지역특화형 비자 운영은 2026년까지 다년도(2년)로 진행되고 85개 기초지자체가 참여, 지역특화 우수인재(F-2-R) 총 5,072명을 배정할 계획이며, 주요 개선사항은 다음과 같습니다. 대상지역 확대 및 쿼터 배정 방식 개선 ○ 기존 인구감소지역(89개)에 추가로 인구감소관심지역(18개)을 포함하여 총 107개 지역으로 확대하고, 지자체가 지역별 특성을 반영하여 해당 지역에 적합한 외국인 유치 정책을 수립할 수 있도록 할 예정입니다. ○ 또한, 대상지역 선정을 위한 공모는 폐지하고, 지자체별로 제출한 사업계획서와 전년도 실적 평가를 바탕으로 지자체에서 신청한 지역특화 우수인재(F-2-R) 5,156명 중 5,072명(배정률 98.3%)을 배정